In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

kopma_df = pd.read_csv(r'../input/connection-loss-data/connectionlossdata.csv', header = 0, index_col = 0, low_memory=False)

kopma_df['Kopma'] = kopma_df['Kopma'].fillna('Saglam')
kopma_df.head()

kopma_df.iloc[:,6:] = kopma_df.iloc[:,6:].apply(LabelEncoder().fit_transform)

#Loading the dataset
X = kopma_df.drop("Kopma",1)   #Feature Matrix
y = kopma_df["Kopma"]          #Target Variable

#Adding constant column of ones, mandatory for sm.OLS model
X_1 = sm.add_constant(X)
#Fitting sm.OLS model
model = sm.OLS(y,X_1).fit()
model.pvalues

In [ ]:
#Backward Elimination
cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

In [ ]:
#wanted_columns = selected_features_BE 
wanted_columns = ['TOPLAMSAAT', 'SORUNSAYISINET', 'PortErrorCount', 'LostCarrierCount', 'GecenAyPortErrorSayisi', 'GecmisFark', 'POSSEHIR', 'POSSEMT', 'SANTRALADI', 'Kopma']
kopma_df = kopma_df[wanted_columns]
kopma_df.head(50)
#kopma_df.to_csv('mycsvfile.csv',index=False)


In [ ]:
from collections import Counter

ko_df = pd.read_csv(r'./mycsvfile.csv', low_memory=False)
ko_df 

Counter(ko_df['Kopma'])

In [ ]:
ko_df.loc[ko_df['Kopma']>0, 'Kopma'] = 1
Counter(ko_df['Kopma'])

In [ ]:
X = ko_df.iloc[:,0:8]
y = ko_df.iloc[:,-1]
X,y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
Counter(y_test)

In [ ]:
# import library
from imblearn.under_sampling import RandomUnderSampler
from sklearn import datasets
import collections

rus = RandomUnderSampler(sampling_strategy={
        0: 1353,
        1: 1353,
    },random_state=42, replacement=True)# fit predictor and target variable
x_rus, y_rus = rus.fit_resample(X_train, y_train)

print('original dataset shape:', Counter(y_train))
print('Resample dataset shape', Counter(y_rus))

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
classifier = RandomForestClassifier(n_estimators=200) 
classifier.fit(x_rus[:10000],y_rus[:10000])

y_pred = classifier.predict(X_test)

from sklearn import metrics 
import numpy as np 
classifier.score(X_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 6, weights = 'distance') 
classifier.fit(x_rus[:10000],y_rus[:10000])

y_pred = classifier.predict(X_test)

from sklearn import metrics 
import numpy as np 
classifier.score(X_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm